In [11]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_1_1.csv')
df['conventional'] = df['conventional'].str.replace(r'[^0-9]', '', regex=True).replace('', np.nan)
df['conventional'] = df['conventional'].astype(float)
print(round(df['conventional'].quantile(0.75) - df['conventional'].quantile(0.25),0))

df['ratio'] = df['minority'] / df['poverty']
print(round(df[(df['ratio'] > 2) & (df['city'] == 'state')]['crime'].mean(),0))

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_1_2.csv')
df['date'] = df['year'].astype(str) + df['month'].astype(str).str.zfill(2)
df['date'] = pd.to_datetime(df['date'], format='%Y%m')
print(round(df[df.date > '2016-08']['actual'].mean(),0))

9.0
61.0
58.0


In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_2_test.csv')

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

key = test['ID']
y_train = train['target']
y_test = test['target']
x_train = train.drop(['level_0', 'ID', 'target'], axis=1)
x_test = test.drop(['level_0', 'ID', 'target'], axis=1)

co = pd.concat([x_train, x_test], axis=0)
st = StandardScaler()
mm = MinMaxScaler()
co[co.columns] = st.fit_transform(co[co.columns])
#co[co.columns] = mm.fit_transform(co[co.columns])

x_train = co.iloc[:len(x_train),:]
x_test = co.iloc[len(x_train):,:]

x_t, x_va, y_t, y_va = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

model = RandomForestClassifier(
    n_estimators = 1000,
    max_depth = 5,
    min_samples_split = 4,
    min_samples_leaf = 3,
    random_state = 42
)

model.fit(x_t, y_t)
pred = model.predict_proba(x_va)[:,1]
print('AUC:', roc_auc_score(y_va, pred))

result = pd.DataFrame()
result['ID'] = key
result['pred'] = model.predict_proba(x_test)[:,1]
print(result.head(2))
#result.to_csv('result.csv', index=False)

AUC: 0.9980657640232108
    ID      pred
0  421  0.464132
1   47  0.227691


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_3_1.csv')
print(round(df.study_hours.mean(),2), round(df.study_hours.std(),2), round(df.exam_scores.mean(),2), round(df.exam_scores.std(),2))
from scipy.stats import pearsonr
stats, p = pearsonr(df.study_hours, df.exam_scores)
print(round(stats, 3))
print(round(p, 3))
if p < 0.05:
    print('기각')
else:
    print('채택')

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_3_2.csv')
print(df.head(2))
from scipy.stats import f_oneway
print('A', round(df[df['campaign'] == 'A'].satisfaction_score.mean(), 2), round(df[df['campaign'] == 'A'].satisfaction_score.std(), 2))
print('B', round(df[df['campaign'] == 'B'].satisfaction_score.mean(), 2), round(df[df['campaign'] == 'B'].satisfaction_score.std(), 2))
print('C', round(df[df['campaign'] == 'C'].satisfaction_score.mean(), 2), round(df[df['campaign'] == 'C'].satisfaction_score.std(), 2))

st, p = f_oneway(df[df['campaign'] == 'A'].satisfaction_score, df[df['campaign'] == 'B'].satisfaction_score, df[df['campaign'] == 'C'].satisfaction_score)
print(round(st, 3))
print(round(p, 3))
if p < 0.05:
    print('기각')
else:
    print('채택')

5.28 2.27 74.79 8.76
-0.059
0.683
채택
  campaign  satisfaction_score
0        A           87.640523
1        A           74.001572
A 71.41 11.37
B 74.79 8.76
C 82.57 10.43
15.606
0.0
기각
